In [1]:
# Python script to do the analysis based on the account table
import openpyxl
import numpy as np
import pandas as pd
import os
from util import rs_conn, mail
import time
import datetime
import logging

In [2]:
# Get the redshift user name and password
rs_user = os.getenv('RS_USER')
rs_pass = os.getenv('RS_PASS')

# Connect to the Redshift
rs = rs_conn.RS_CONN(rs_user=rs_user, rs_pass=rs_pass)
rs.set_Conn()

In [11]:
acct_engagement_pull_all_cycle="""
select 
        a.cycle_end_dt
        , a.vintage
        ,sum(case when a.acct_status_desc = 'Active' then 1 else 0 end) as active_acct_cnt 
        ,sum(a.stmt_charged_interest_amt) as assessed_interest 
        ,sum(b.stmt_paid_interest_amt) as collected_interest 
        ,sum(b.stmt_paid_interest_amt) - sum(b.stmt_tot_refund_interests_amt) as adj_interest_revenue_amt
        
        ,sum(a.stmt_charged_fee_amt) as assessed_fee 
        ,sum(b.stmt_paid_fee_amt) - sum(b.stmt_tot_refund_fee_amt) as fee_revenue_amt 
        
        ,sum(case when a.stmt_num=1 then a.open_fee_amt else 0 end) as open_fee_amt_
        ,sum(a.stmt_tot_interchange_fee_amt) as interchange_revenue_amt
        ,sum(coalesce(a.service_pmt_commission_amt,0)) as service_pmt_revenue_amt
        ,coalesce(adj_interest_revenue_amt+fee_revenue_amt+open_fee_amt_+interchange_revenue_amt+service_pmt_revenue_amt,0) as total_revenue
        
from ua.wen_stmt_acct_table_v3 a
left outer join ua.wen_stmt_acct_table_v3 b 
    on a.external_acct_id = b.external_acct_id
    and a.cycle_end_dt = b.cycle_start_dt - 1
where a.activation_dt is not null
and a.acct_status_desc <> 'Inactive'
group by 1,2
order by 1
;
"""
df=rs.get_result_df(sql_query=acct_engagement_pull_all_cycle)

In [32]:
df = pd.read_csv("df.csv")

In [30]:
df.columns

Index(['Unnamed: 0', 'cycle_end_dt', 'vintage', 'active_acct_cnt',
       'assessed_interest', 'collected_interest', 'adj_interest_revenue_amt',
       'assessed_fee', 'fee_revenue_amt', 'open_fee_amt_',
       'interchange_revenue_amt', 'service_pmt_revenue_amt', 'total_revenue',
       'arpu'],
      dtype='object')

In [28]:
#df["total_revenue"] = df["adj_interest_revenue_amt"] + df["fee_revenue_amt"] + df["open_fee_amt"] + df["interchange_revenue_amt"] + df["service_pmt_revenue_amt"]
df["arpu"] = df["total_revenue"]/df["active_acct_cnt"]

In [33]:
total_revenue = pd.pivot_table(df,values='total_revenue',index='cycle_end_dt',columns='vintage  ',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
total_revenue.to_csv("total_revenue.csv")

In [21]:
arpu = pd.pivot_table(df,values='arpu',index='vintage',columns='cycle_end_dt',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
arpu.to_csv("arpu.csv")

KeyError: 'arpu'